# GRASS GIS LiDAR Derived Hydrography Flowlines

This notebook prototypes development of LiDAR elevation derived flow path data in [GRASS GIS]( https://grass.osgeo.org).

## Install GRASS GIS

Run this notebook within the GRASS GIS environment.  If GRASS is not currently installed download from: [https://grass.osgeo.org/download/](https://grass.osgeo.org/download/).

## Checking the python environment

In [ ]:
import sys
print (sys.version)

## Checking the file path

Recall this check from notebook No. 1.

In [ ]:
%%bash
pwd

In [ ]:
!pwd

In [ ]:
!echo %cd%

## Create a new GRASS GIS Project

Creating a new [GRASS GIS]( https://grass.osgeo.org) project.  For LiDAR derived hydrography the project will be conducted in the same Coordinate Reference System (CRS) of the LiDAR based digital elevation model (DEM) downloaded in the original product resolution (OPR) folder.  Flow path analysis may be developed from pre-existing DEMs or DEMs generated from point cloud data.

The `ls` command provides a directory listing at the present working directory (pwd). At the beginning of many of the code cells below, there are commented bash lines.  These have been substituted with python lines to emulate the corresponding bash functions.  On macOS the commenting could be swapped and run if desired.  In some instances where there are inherent Windows calls, for example to the GRASS .bat executable, the commenting should be swapped in order to successfully run those unique cells on macOS.

In [ ]:
# !ls 

import os

def ls (directory):
    try:
        # List the contents of the current directory
        directory_contents = os.listdir(directory)
        
        # Print each item in the directory
        for item in directory_contents:
            print(item)
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
ls('.') # List the contents of the current directory

Next, double check the CRS of the DEM data to ensure consistent setup of the GRASS project using the same (CRS).

In [ ]:
# !ls ./OPR/*.tif

ls('./OPR')

Selecting one of the GeoTIFF files, read the coordinate system with [gdalinfo](https://gdal.org/programs/gdalinfo.html#gdalinfo).

In [ ]:
# %%bash
# gdalinfo ./OPR/D03_37_10875003_20160228.tif
!gdalinfo ./OPR/D03_37_10875003_20160228.tif

Based on the above output, the command below will create (with the -`c` flag) a GRASS GIS project in North Carolina State Plane projection with US Survey Foot (USFT) units using the corresponding code: [EPSG:6543](https://epsg.org/crs_6543/NAD83-2011-North-Carolina-ftUS.html).  The following GRASS GIS command builds the empty GRASS project at the local home directory path (and exits with the -`e` flag).  

For operating systems other than Windows, swap the commenting to create the GRASS GIS project.

In [ ]:
# %%bash
# grass -c EPSG:6543 -e ~/grassdata/GRASS_6543/
# !grass -c EPSG:6543 -e ~/grassdata/GRASS_6543/
!"C:\Program Files\GRASS GIS 8.4\grass84.bat" -c EPSG:6543 -e "%USERPROFILE%\grassdata\GRASS_6543"

In [ ]:
# %%bash
# grass -c EPSG:6543 -e ~/grassdata/GRASS_6543/
# !grass -c EPSG:6543 -e ~/grassdata/GRASS_6543/
# !"C:\Program Files\GRASS GIS 8.4\grass84.bat" -c EPSG:6543 -e ~/grassdata/GRASS_6543/
# !"C:\Program Files\GRASS GIS 8.4\grass84.bat" --version 
# !"C:\Program Files\GRASS GIS 8.4\grass84.bat" -c EPSG:6543 -e ./grassdata/GRASS_6543/ --verbose
# import os
# # import grass_session
# import grass.script as gs
# import grass.jupyter as gj

# # Get the user's home directory in a cross-platform way
# home = os.path.expanduser("~")

# # Construct the path
# grass_data_path = os.path.join(home, "grassdata", "GRASS_6543")

# # Ensure the directory exists
# os.makedirs(grass_data_path, exist_ok=True)

# # Initialize GRASS session
# grass_session.Session()
# session = gj.init(grass_data_path, "PERMANENT")

# # Print region info to confirm initialization
# print(gs.read_command("g.region", flags="p"))


import os
import grass.script as gs
import grass.jupyter as gj

# Get the user's home directory in a cross-platform way
home = os.path.expanduser("~")

# Construct the path to the GRASS database
grass_data_path = os.path.join(home, "grassdata")

# Location name
location = "GRASS_6543"

# Mapset name
mapset = "PERMANENT"

# Initialize GRASS session
try:
    session = gj.init(grass_data_path, location, mapset)
    print("Session initialized successfully")
    
    # Print GRASS environment info
    print("\nGRASS environment:")
    print(gs.gisenv())
    
    # Print region info
    print("\nRegion info:")
    print(gs.read_command("g.region", flags="p"))
except Exception as e:
    print(f"Error initializing GRASS session: {e}")
    
    # Additional error information
    print("\nCurrent working directory:", os.getcwd())
    print("GRASS_DATA_PATH:", grass_data_path)
    print("Location:", location)
    print("Mapset:", mapset)

Information about running GRASS GIS in jupyter notebooks is available at the following [link](https://grass.osgeo.org/grass83/manuals/libpython/index.html).  The next cell will initiate the GRASS GIS project just created and import required libraries.

Check the current GRASS GIS environment and list the mapset, database, location and projection information.  

In [ ]:
# %%bash
# g.gisenv
# g.proj -p

!g.gisenv
!g.proj -p

## Import / Create LiDAR DEM data

Data (DEM / LiDAR) for this exercise is available at the NOAA Digital Coast bulk download site:  

- DEM:  [https://chs.coast.noaa.gov/htdata/raster2/elevation/NorthCarolina_DEM_2015P3_6205/](https://chs.coast.noaa.gov/htdata/raster2/elevation/NorthCarolina_DEM_2015P3_6205/)
- LiDAR:  [https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/6209/index.html](https://noaa-nos-coastal-lidar-pds.s3.amazonaws.com/laz/geoid18/6209/index.html)

Import the DEM with GRASS GIS [r.import](https://grass.osgeo.org/grass84/manuals/r.import.html).

In [ ]:
!r.import in=Reference/NC_P3_2015_TEM_Coleridge_SE_opr.tif out=NC_P3_2015_TEM_Coleridge_SE_opr --overwrite

In [ ]:
import grass.script as gs

# List raster maps to check if NC_P3_2015_TEM_Coleridge_SE_opr is present
print("Raster maps:")
print(gs.list_strings(type='raster'))

# Get information about the imported raster
print("\nRaster info:")
print(gs.raster_info('NC_P3_2015_TEM_Coleridge_SE_opr'))

# Set region to the imported raster and print region info
gs.run_command('g.region', raster='NC_P3_2015_TEM_Coleridge_SE_opr')
print("\nRegion info:")
print(gs.read_command('g.region', flags='p'))

The GRASS GIS command [g.list](https://grass.osgeo.org/grass84/manuals/g.list.html) can list available maps.  It may be necessary to interrupt the cell.

In [ ]:
!g.list rast 

The next cell sets the GRASS GIS [computational region](https://grasswiki.osgeo.org/wiki/Computational_region) to the imported raster dimensions.

In [ ]:
!g.region -p rast=NC_P3_2015_TEM_Coleridge_SE_opr

For visual feedback, the next cell displays the imported map.

In [ ]:
# Define Grass Jupyter Map 
NC_P3_2015_TEM_opr_map = gj.Map()
# Add raster data and legend to the map
NC_P3_2015_TEM_opr_map.d_rast(map="NC_P3_2015_TEM_Coleridge_SE_opr")
NC_P3_2015_TEM_opr_map.d_legend(raster="NC_P3_2015_TEM_Coleridge_SE_opr", color="red")
# Display the map
NC_P3_2015_TEM_opr_map.show()

## Map Unit Analysis

On USGS QL2-based projects, BHI traditional data processing mode is to conduct analysis at the USGS 7.5 minute quadrangle series level.  In this exercise, the 3.75 minute quarter quadrangle series is utilized as the atomic unit for processing and import a vector cell map for this TEM area of interest (AOI).

In [ ]:
!v.import in=Reference/cellgrid_3_75minute_tem.gpkg out=cellgrid_3_75minute_tem

In [ ]:
# Define Grass Jupyter Map 
NC_P3_2015_TEM_opr_map = gj.Map()
# Add raster data and legend to the map
NC_P3_2015_TEM_opr_map.d_rast(map="NC_P3_2015_TEM_Coleridge_SE_opr")
NC_P3_2015_TEM_opr_map.d_legend(raster="NC_P3_2015_TEM_Coleridge_SE_opr", color="red")
NC_P3_2015_TEM_opr_map.d_vect(map="cellgrid_3_75minute_tem", fill="none", attribute_column="CELL_NAME", xref="center", label_size=12)
# Display the map
NC_P3_2015_TEM_opr_map.show()

List the cellgrid vector table columns with [db.columns](https://grass.osgeo.org/grass84/manuals/db.columns.html) and [v.db.select](https://grass.osgeo.org/grass84/manuals/v.db.select.html).

In [ ]:
!db.columns table=cellgrid_3_75minute_tem

In [ ]:
# !v.db.select map=cellgrid_3_75minute_tem column=CELL_NAME | tail -n +2 | sed 's/ /_/g'

import grass.script as gs
import re

def process_cell_names():
    # Get the data from v.db.select
    data = gs.read_command('v.db.select', map='cellgrid_3_75minute_tem', columns='CELL_NAME')
    
    # Split the output into lines
    lines = data.strip().split('\n')
    
    # Skip the header (first line)
    lines = lines[1:]
    
    # Replace spaces with underscores and remove trailing underscores
    processed_lines = [re.sub(r'\s+', '_', line).rstrip('_') for line in lines]
    
    return processed_lines

Output the above command as a file list.

In [ ]:
# !v.db.select map=cellgrid_3_75minute_tem column=CELL_NAME | tail -n +2 | sed 's/ /_/g' > Reference/CellGrid_3_75Minute_TEM.txt

# Set the output file path
output_file = os.path.join('Reference', 'CellGrid_3_75Minute_TEM.txt')
result = process_cell_names()

# Write to file
with open(output_file, 'w') as f:
    for line in result:
        f.write(line + '\n')

print(f"Data has been written to {output_file}")

In [ ]:
# !cat Reference/CellGrid_3_75Minute_TEM.txt

with open(output_file, 'r') as f:
    for line in f:
        print(line)

Extract 3.75 minute quadrangle units for processing with a shell script.  The analog python script is also provided for Windows interoperability.

In [ ]:
# !sh ./make_quad_extract.sh
!python make_quad_extract.py

Again, [g.list](https://grass.osgeo.org/grass84/manuals/g.list.html) can list available vector maps in the following case.

In [ ]:
!g.list vect

Individual steps follow in the next cells to demonstrate processing using the Coleridge SE 3.75 minute unit, starting with setting the region with [g.region](https://grass.osgeo.org/grass84/manuals/g.region.html).

In [ ]:
!g.region -pa res=3.125 vect=Coleridge_SE_Mask

In traditional BHI flow line processing, the region has normally been extended (by 3600 feet) to ensure consistent map boundary edge matching.  For this exercise, given limited DEM coverage, unit boundary analysis regions will be extended by 600 feet at the native resolution of the DEM data -- 3.125 feet.  Traditional processing for QL2 has normally operated at 2 foot pixel resolution and for QL1 at 1 foot pixel resolution.

In [ ]:
!g.region -p -a n=n+600 s=s-600 w=w-600 e=e+600

## Hydro Flow Line Processing.

In [ ]:
!r.watershed --overwrite elevation=NC_P3_2015_TEM_Coleridge_SE_opr accumulation=Coleridge_SE_ACC

In the next cell the computational region is reduced by 100 feet. 

In [ ]:
!g.region -p -a n=n-100 s=s+100 w=w+100 e=e-100

The flow accumulation raster is output in the next cell with [r.out.gdal](https://grass.osgeo.org/grass84/manuals/r.out.gdal.html) and can be loaded and visualized in other GIS packages such as QGIS.

In [ ]:
!r.out.gdal -f --overwrite input=Coleridge_SE_ACC output=Reference/Coleridge_SE_ACC.tif format=GTiff type=Float64 createopt=COMPRESS=LZW,PREDICTOR=3,TILED=YES,BLOCKXSIZE=128,BLOCKYSIZE=128,BIGTIFF=YES overviews=5

The flow line network is then extracted with [r.stream.extract](https://grass.osgeo.org/grass84/manuals/r.stream.extract.html); note the threshold value.

In [ ]:
!r.stream.extract elevation="NC_P3_2015_TEM_Coleridge_SE_opr" accumulation=Coleridge_SE_ACC direction=Coleridge_SE_Direction threshold=1000 stream_rast=Coleridge_SE_Stream --overwrite

The next cells add stream ordering with the GRASS GIS Add-on [r.stream.order](https://grass.osgeo.org/grass84/manuals/addons/r.stream.order.html).

Install the Add-on.

In [ ]:
!r.stream.order

In [ ]:
!g.extension extension=r.stream.order

In [ ]:
!r.stream.order --overwrite stream_rast=Coleridge_SE_Stream direction=Coleridge_SE_Direction elevation=NC_P3_2015_TEM_Coleridge_SE_opr accumulation=Coleridge_SE_ACC stream_vect=Coleridge_SE_Stream strahler=Coleridge_SE_Strahler horton=Coleridge_SE_Horton shreve=Coleridge_SE_Shreve hack=Coleridge_SE_Hack topo=Coleridge_SE_Topo

Generalization with [v.generalize](https://grass.osgeo.org/grass84/manuals/v.generalize.html) using the GRASS GIS snakes algorithm.

In [ ]:
!v.generalize --overwrite input=Coleridge_SE_Stream output=Coleridge_SE_Order_Smooth method=snakes threshold=2

Below [v.out.ogr](https://grass.osgeo.org/grass84/manuals/v.out.ogr.html) writes both point-based and line-based flow line features. 

In [ ]:
!v.out.ogr --overwrite input=Coleridge_SE_Order_Smooth output=Reference/Coleridge_SE_Order_Smooth.sqlite format=SQLite

In [ ]:
# Define Grass Jupyter Map 
NC_P3_2015_TEM_opr_map = gj.Map()
# Add raster data and legend to the map
NC_P3_2015_TEM_opr_map.d_rast(map="NC_P3_2015_TEM_Coleridge_SE_opr")
NC_P3_2015_TEM_opr_map.d_legend(raster="NC_P3_2015_TEM_Coleridge_SE_opr", color="red")
NC_P3_2015_TEM_opr_map.d_vect(map="Coleridge_SE_Order_Smooth", color="blue")
# Display the map
NC_P3_2015_TEM_opr_map.show()

For ["InteractiveMap"](https://onlinelibrary.wiley.com/doi/full/10.1111/tgis.13031) display in GRASS GIS in may be necessary to install ipyleaflet.

In [ ]:
pip install ipyleaflet

It may be necessary to restart the kernel and if running the notebook from VSCode, the application may prompt to accept Widget installation upon execution of the next cell.

In [ ]:
# Create an instance of InteractiveMap
map_instance = gj.InteractiveMap(width=800, height=600)

# Print out the available methods and attributes
print(dir(map_instance))

Below v.extract is run to separate the lines in an attempt to make the data lighter for InteractiveMap viewing.

In [ ]:
!v.extract --overwrite input=Coleridge_SE_Order_Smooth output=Coleridge_SE_Order_Smooth_line type=line

The InteractiveMap view below may take about a minute to generate give the dataset volume.  In the meantime the GRASS data output with [r.out.gdal](https://grass.osgeo.org/grass84/manuals/r.out.gdal.html) and [v.out.ogr](https://grass.osgeo.org/grass84/manuals/v.out.ogr.html) can be viewed in QGIS.

In [ ]:
import grass.jupyter as gj

# Define Grass Jupyter Map with interactive features
NC_P3_2015_TEM_opr_map = gj.InteractiveMap()  # Adjust size as needed

# Add raster data to the map
NC_P3_2015_TEM_opr_map.add_raster("NC_P3_2015_TEM_Coleridge_SE_opr")

# Add vector data (streams) to the map
NC_P3_2015_TEM_opr_map.add_vector("Coleridge_SE_Order_Smooth_line", color="blue", type="line", where="strahler >= 5")

# Add layer control
NC_P3_2015_TEM_opr_map.add_layer_control()

# Display the map
NC_P3_2015_TEM_opr_map.show()

## Optional:  Geomorphic Processing

As for the Hydro Flowline Processing, set the region to the Coleridge SE Mask.

In [ ]:
!g.region -pa res=3.125 vect=Coleridge_SE_Mask

Expand the region.

In [ ]:
!g.region -p -a n=n+600 s=s-600 w=w-600 e=e+600

Geomorphic landform identification with [r.geomorphon](https://grass.osgeo.org/grass84/manuals/r.geomorphon.html) can be instrumental in culvert detection when hydro-enforcement is required. This next cell takes about ~6 minutes to run with the search settings below. 

In [ ]:
# %%bash
# START=$(date +%s);
# sleep 1; 
# echo $START
# r.geomorphon elevation=NC_P3_2015_TEM_Coleridge_SE_opr forms=Coleridge_SE_geomorph search=40 skip=5 flat=6 dist=6 --overwrite
# END=$(date +%s);
# echo ----- $((END-START)) seconds -----

import time

start_time = time.time()

gs.run_command('r.geomorphon', 
               elevation='NC_P3_2015_TEM_Coleridge_SE_opr',
               forms='Coleridge_SE_geomorph',
               search=40,
               skip=5,
               flat=6,
               dist=6,
               overwrite=True)

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

Optionally, mode filtering of the geomorphic landform data with [r.neighbors](https://grass.osgeo.org/grass84/manuals/r.neighbors.html) can help with interpretation depending on the dataset.

In [ ]:
!r.neighbors in=Coleridge_SE_geomorph out=Coleridge_SE_geomorph_mode3 method=mode nprocs=6 --overwrite

The next cell reduces the region to match the flow accumulation output.

In [ ]:
!g.region -p -a n=n-100 s=s+100 w=w+100 e=e-100

The geomorphic landforms are then exported with [r.out.gdal](https://grass.osgeo.org/grass84/manuals/r.out.gdal.html) and can be viewed in QGIS.

In [ ]:
!r.out.gdal -f --overwrite input=Coleridge_SE_geomorph output=Reference/Coleridge_SE_geomorph.tif format=GTiff type=Byte createopt=COMPRESS=LZW,PREDICTOR=2,TILED=YES,BLOCKXSIZE=128,BLOCKYSIZE=128,BIGTIFF=YES overviews=5